### Install necessary packages

In [ ]:
pip install -r requirements.txt

In [42]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)
import nest_asyncio

nest_asyncio.apply()

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core import PromptTemplate
from llama_index.core.memory import ChatMemoryBuffer

In [20]:
import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Set embedding model

In [ ]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

### Set LLM with Ollama (tinyllama)

In [22]:
llm = Ollama(model="tinyllama:latest", request_timeout=120.0)
Settings.llm = llm
Settings.chunk_size = 512

### Test LLM

In [ ]:
messages = [
    ChatMessage(
        role="system", content="Tu es un Jane Austen"
    ),
    ChatMessage(role="user", content=""),
]
resp = llm.chat(messages)
print(resp)

# Load stores and indexes

### Vector

In [53]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

In [ ]:
simple_index = load_index_from_storage(vector_storage_context)

### Property graph

In [ ]:
pg_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="pg_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="pg_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="pg_graph"),
)
print(pg_storage_context)

In [ ]:
pg_index = load_index_from_storage(pg_storage_context)

##### Visualize property graph

In [ ]:
pg_index.property_graph_store.save_networkx_graph(
    name="PgGraph.html"
)

# Producing analytical content with queries

The following blocs of code will produce several queries and use them to generate an analytical content. Each query produces one output stored into memory. The outputs are then bundled together into final content



### GraphRAG using Property Graph

In [ ]:
memory = ChatMemoryBuffer.from_defaults(token_limit=10000)
chat_engine = pg_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        ""
    ),
    verbose=False,
)

In [ ]:
chat_engine.reset()

In [ ]:
response_stream = chat_engine.stream_chat("""
                                                
                                                """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"**/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 

### RAG using vector store and index

In [55]:
custom_query_template = PromptTemplate(
    "Nous avons fourni des informations contextuelles ci-dessous. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "En utilisant ces informations, veuillez répondre à la question suivante : {query_str}\n"
)


In [56]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
       """ 
       Tu es une femme du XVIIIème siècle, tu es une écrivaine anglaise. Tu es connue pour ton roman Orgueil et Préjugés.
       """
    ),
    text_qa_template=custom_query_template,
    verbose=False,
    similarity_top_k=4
    
)

In [ ]:
response_stream = vector_chat_engine.stream_chat("""Dans la société du XVIIIème siècle, quelles sont les qualités des hommes ? des femmes ? """)

In [ ]:
generate = response_stream.print_response_stream()

In [ ]:
# Afficher les chunks utilisés
for node in response_stream.source_nodes:
    print(node.node.text)
    

In [ ]:
query = "Dans la société du XVIIIème siècle, quelles sont les qualités des hommes ? des femmes ? Répond en français"
query_engine = simple_index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=4,
)

response = query_engine.query(query)



In [ ]:
print(response)

In [ ]:
# Afficher les chunks utilisés
for node in response.source_nodes:
    print(node.node.text)
    print("---")

In [62]:
vector_chat_engine.reset()